Input data

In [ ]:
# Specify the input folder containing NDWI GeoTIFF files and the output folder
input_folder = r"D:\River connectivity\Whole Danube Reach in Hungary\NDWI input\clear_NDWI"
output_folder = r"D:\River connectivity\Whole Danube Reach in Hungary\Water mask output\clear mask"

#The block size and offset can be fine tuned to achive the best results. It should be changed the following code at all lines

Processing code

In [ ]:
import os
import rasterio
import numpy as np
from skimage.filters import threshold_local

# Function to read the NDWI GeoTIFF image
def read_ndwi_image(image_path):
    with rasterio.open(image_path) as src:
        ndwi = src.read(1)  # Reading the first band (assume NDWI is in a single band)
        profile = src.profile  # Get the image metadata/profile
        ndwi[ndwi == src.nodata] = np.nan  # Replace NoData values with NaN
    return ndwi, profile

# Function to handle NaN values (replace them with a valid number)
def handle_nan_values(ndwi_image, fill_value=0):
    ndwi_clean = np.nan_to_num(ndwi_image, nan=fill_value)  # Replace NaN with a fill value (e.g., 0)
    return ndwi_clean

# Function to apply only local thresholding
def apply_local_threshold(ndwi_image, block_size=4999, offset=-0.09):
    # Step 1: Clean the NDWI image by handling NaN values
    ndwi_clean = handle_nan_values(ndwi_image)
    
    # Step 2: Normalize the NDWI image to the range 0-1
    ndwi_normalized = (ndwi_clean - np.nanmin(ndwi_clean)) / (np.nanmax(ndwi_clean) - np.nanmin(ndwi_clean))
    
    # Step 3: Local thresholding
    local_thresh = threshold_local(ndwi_normalized, block_size=block_size, offset=offset)
    binary_local = ndwi_normalized > local_thresh
    
    return binary_local

# Function to export the binary mask to a GeoTIFF
def export_binary_mask(mask, profile, output_path):
    # Update profile to match the binary mask data type (uint8)
    profile.update(dtype=rasterio.uint8, count=1, nodata=0)
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(mask.astype(rasterio.uint8), 1)  # Write the mask as uint8 (0 and 1)

# Function to process all images in a folder
def process_ndwi_folder(input_folder, output_folder, block_size=4999, offset=-0.09):
    # Loop through all .tif files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):
            image_path = os.path.join(input_folder, filename)
            print(f"Processing: {filename}")
            
            # Step 1: Read the NDWI image
            ndwi_image, profile = read_ndwi_image(image_path)
            
            # Step 2: Apply only local thresholding
            binary_local = apply_local_threshold(ndwi_image, block_size, offset)
            
            # Extract the date from the filename
            date = filename.split('_')[1].split('.')[0]
            
            # Step 3: Export the local threshold mask to a GeoTIFF file
            output_path_local = os.path.join(output_folder, f"watermask_local_{date}.tif")
            export_binary_mask(binary_local, profile, output_path_local)
            print(f"Exported local Otsu water mask to: {output_path_local}")



# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all NDWI images in the folder and export water masks
process_ndwi_folder(input_folder, output_folder, block_size=4999, offset=-0.09)

print("Processing completed.")
